<a href="https://colab.research.google.com/github/mohripan/LLM-repo/blob/main/GPT2_PreTrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch] --upgrade --force-reinstall
!pip install datasets

  Using cached transformers-4.34.1-py3-none-any.whl (7.7 MB)
  Using cached filelock-3.13.1-py3-none-any.whl (11 kB)
  Using cached huggingface_hub-0.18.0-py3-none-any.whl (301 kB)
  Using cached numpy-1.26.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (705 kB)
  Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (773 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)
  Using cached safetensors-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached accelerate-0.24.1-py3-none-any.whl (261 kB)
  Using cached psutil-5.9.6-cp3

In [2]:
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)

from datasets import load_dataset

import torch

In [3]:
# Load the tokenizer, model, and dataset
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
dataset = load_dataset('glue', 'sst2')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def tokenize_function(examples):
    tokenizer_output = tokenizer(
        examples['sentence'],
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="pt"  # Return PyTorch tensors
    )

    return {
        'input_ids': tokenizer_output['input_ids'],
        'attention_mask': tokenizer_output['attention_mask'],
        'labels': examples['label']
    }

In [5]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [12]:
# Prepare data format for PyTorch
small_train_dataset = tokenized_datasets['train'].shuffle(seed=42)
small_test_dataset = tokenized_datasets['validation'].shuffle(seed=42)

In [13]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [14]:
# Define training arguments and initialize Trainer
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy='epoch',
    output_dir='./results',
    logging_dir='./logs',
    logging_steps=10,
    do_train=True,
    do_eval=True,
    no_cuda=False,
    load_best_model_at_end=True,
    save_strategy='epoch',
    report_to='tensorboard',
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    tokenizer=tokenizer,
)

In [15]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.490300,0.612710
2,0.164600,0.414620
3,0.086600,0.416264


TrainOutput(global_step=25257, training_loss=0.19885415897247094, metrics={'train_runtime': 4922.6094, 'train_samples_per_second': 41.045, 'train_steps_per_second': 5.131, 'total_flos': 1.329019985058048e+16, 'train_loss': 0.19885415897247094, 'epoch': 3.0})

In [11]:
# Evaluate the model
results = trainer.evaluate()
print(results)

{'eval_loss': 0.3133249282836914, 'eval_runtime': 3.6466, 'eval_samples_per_second': 137.113, 'eval_steps_per_second': 17.276, 'epoch': 3.0}
